<a href="https://colab.research.google.com/github/IT21208294/Shortest-Path/blob/Tharushi/SLTshortestPath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simanneal

In [2]:
import pandas as pd
import numpy as np
from simanneal import Annealer
from math import radians, sin, cos, sqrt, atan2
import urllib.parse

In [3]:
# Load CSV data
data = pd.read_csv('/content/locations.csv.txt')

In [4]:
# Extract location names, latitudes, and longitudes
location_names = data['Name'].tolist()
latitudes = data['Latitude'].tolist()
longitudes = data['Longitude'].tolist()

In [5]:
# Number of locations
n_locations = len(location_names)

In [6]:
# Define a function to calculate the Haversine distance between two points
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [7]:
# Calculate distance matrix
distance_matrix = np.zeros((n_locations, n_locations))
for i in range(n_locations):
    for j in range(n_locations):
        if i != j:
            distance_matrix[i, j] = haversine_distance(latitudes[i], longitudes[i], latitudes[j], longitudes[j])

In [8]:
# Function to set the start node
def set_start_node(route, start_node):
    """Set the start node of the route and move it to the beginning."""
    if start_node != 0:
        route.remove(start_node)
        route.insert(0, start_node)
    return route

In [9]:
# Define a function to calculate the total distance for a given route
def calculate_distance(route):
    distance = 0
    for i in range(len(route) - 1):
        distance += distance_matrix[route[i], route[i + 1]]
    distance += distance_matrix[route[-1], route[0]]
    return distance

In [10]:
# Define a custom simulated annealing class for TSP
class TSPAnnealer(Annealer):
    def __init__(self, initial_route):
        self.route = initial_route
        super().__init__(initial_route)

    def move(self):
        # Swap two cities in the route
        i = np.random.randint(1, len(self.route) - 1)
        j = np.random.randint(1, len(self.route) - 1)
        self.route[i], self.route[j] = self.route[j], self.route[i]

    def energy(self):
        # Return the total distance of the current route
        return calculate_distance(self.route)

In [11]:
# Set the initial route
initial_route = list(range(n_locations))

In [12]:
# Specify the start node
start_node = 0

In [13]:
# Update the initial route to start from the specified start node
initial_route = set_start_node(initial_route, start_node)

In [14]:
# Create an instance of the TSPAnnealer with the initial route
annealer = TSPAnnealer(initial_route)

In [15]:
# Run the simulated annealing algorithm
annealer.auto(minutes=5, steps=1000)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


{'tmax': 81.0,
 'tmin': 1.0000000000000001e-21,
 'steps': 4400000,
 'updates': 100}